### __Load Environment Variables__

In [32]:
from dotenv import load_dotenv
load_dotenv()

True

### __State__

In [33]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

### __Tools__

In [34]:
from langchain_tavily import TavilySearch

tools = [TavilySearch(max_results=1)]

In [35]:
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = "Bạn là một trợ lý hữu ích."
agent_executor = create_react_agent(llm, tools, prompt=prompt)

In [36]:
agent_executor.invoke({"messages": [("user", "Thời tiết ở Việt Nam như thế nào?")]})

{'messages': [HumanMessage(content='Thời tiết ở Việt Nam như thế nào?', additional_kwargs={}, response_metadata={}, id='7b871a4a-1521-4366-8824-c6b4674f0008'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nzvCtwok2DZT5QJATZok8C0P', 'function': {'arguments': '{"query":"thời tiết Việt Nam","search_depth":"basic"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 785, 'total_tokens': 810, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cw8veLPMBW8MaVpfQuwnh6Rrmdlbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--71ef4fb0-1636-42f7-99fb-08d7870f7e17-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '

### __Planner__

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class Plan(BaseModel):
    """Kế hoạch thực hiện (Plan to follow)."""

    steps: List[str] = Field(
        description="các bước thực hiện cụ thể, cần được sắp xếp theo đúng thứ tự logic"
    )

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Với mục tiêu được đưa ra, hãy lập một kế hoạch đơn giản theo từng bước (step-by-step). \
Kế hoạch này bao gồm các nhiệm vụ riêng lẻ mà nếu được thực hiện chính xác sẽ dẫn đến câu trả lời đúng. \
Đừng thêm bất kỳ bước thừa thãi nào. \
Kết quả của bước cuối cùng phải là câu trả lời cuối cùng cho người dùng. \
Hãy chắc chắn rằng mỗi bước đều chứa đầy đủ thông tin cần thiết - không được bỏ qua bước nào.""",
        ),
        ("placeholder", "{messages}"),
    ]
)

planner = planner_prompt | ChatOpenAI(
    model="gpt-4o-mini", temperature=0
).with_structured_output(Plan)

planner.invoke(
    {
        "messages": [
            ("user", "Thời tiết ở Việt Nam hiện tại thế nào?")
        ]
    }
)

Plan(steps=['Truy cập vào một trang web thời tiết đáng tin cậy hoặc ứng dụng thời tiết trên điện thoại.', 'Tìm kiếm thông tin thời tiết cho Việt Nam hoặc cụ thể hơn là thành phố mà bạn quan tâm.', 'Ghi chú lại các thông tin như nhiệt độ, độ ẩm, tình trạng thời tiết (mưa, nắng, gió,...) và dự báo thời tiết trong những ngày tới.', 'Tóm tắt các thông tin đã ghi chú lại để có cái nhìn tổng quan về thời tiết hiện tại ở Việt Nam.'])

### __Executor__

In [38]:
from typing import Union


class Response(BaseModel):
    """Câu trả lời gửi tới người dùng."""

    response: str


class Act(BaseModel):
    """Hành động cần thực hiện tiếp theo."""

    action: Union[Response, Plan] = Field(
        description="Hành động cần thực hiện. "
        "Nếu bạn muốn trả lời người dùng (đã xong việc), hãy sử dụng `Response`. "
        "Nếu bạn cần tiếp tục sử dụng công cụ để tìm câu trả lời, hãy sử dụng `Plan`."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """Với mục tiêu đã cho, hãy lập một kế hoạch đơn giản từng bước (step-by-step). \
Kế hoạch này bao gồm các nhiệm vụ riêng lẻ mà nếu thực hiện chính xác sẽ dẫn đến câu trả lời đúng. \
Đừng thêm bất kỳ bước thừa thãi nào. \
Kết quả của bước cuối cùng phải là câu trả lời cuối cùng. \
Đảm bảo rằng mỗi bước đều chứa đầy đủ thông tin cần thiết - không được bỏ qua bước nào.

Mục tiêu ban đầu của bạn là:
{input}

Kế hoạch ban đầu của bạn là:
{plan}

Bạn hiện đã hoàn thành các bước sau:
{past_steps}

Hãy cập nhật kế hoạch của bạn cho phù hợp. \
Nếu không cần thực hiện thêm bước nào nữa và bạn đã có thể trả lời người dùng, hãy phản hồi ngay (dùng Response). \
Ngược lại, hãy điền vào kế hoạch tiếp theo (dùng Plan). \
CHỈ THÊM những bước VẪN CẦN phải làm vào kế hoạch. \
ĐỪNG đưa những bước đã làm xong vào kế hoạch mới."""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Act)

### __Graph__

In [39]:
from langgraph.graph import END

async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""Dựa trên kế hoạch tổng thể sau đây:
                        {plan_str}

                        Nhiệm vụ cụ thể của bạn là thực hiện bước số {1}: {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [40]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["agent", END],
)

# This compiles it into a LangChain Runnable,
# meaning you can use it as any other runnable
app = workflow.compile()

In [41]:
app.get_graph(xray=True).draw_mermaid_png(output_file_path="static/plan-and-execute_agent_graph.png")
app.get_graph(xray=True).print_ascii()

                  +-----------+              
                  | __start__ |              
                  +-----------+              
                        *                    
                        *                    
                        *                    
                  +---------+                
                  | planner |                
                  +---------+                
                        *                    
                        *                    
                        *                    
                  +-----------+              
                  | __start__ |              
                  +-----------+..            
                   *             ...         
                 **                 ...      
                *                      ....  
          +-------+                        ..
          | agent |                         .
          +-------+                         .
         *         .              

### __Test__

In [42]:
config = {"recursion_limit": 50}
inputs = {"input": "Con gà có trước hay quả trứng có trước?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

{'plan': ["Xác định câu hỏi: 'Con gà có trước hay quả trứng có trước?'", 'Nghiên cứu về nguồn gốc của gà: Gà là loài động vật có vú, thuộc họ gà (Phasianidae).', 'Tìm hiểu về quá trình sinh sản của gà: Gà đẻ trứng để sinh sản.', 'Xem xét lý thuyết tiến hóa: Gà hiện đại có thể đã phát triển từ một loài chim khác thông qua quá trình tiến hóa.', 'Kết luận rằng quả trứng có thể đã xuất hiện trước con gà, vì tổ tiên của gà đã đẻ trứng trước khi có gà hiện đại.', 'Đưa ra câu trả lời cuối cùng: Quả trứng có trước con gà.']}
{'past_steps': [("Xác định câu hỏi: 'Con gà có trước hay quả trứng có trước?'", 'Câu hỏi "Con gà có trước hay quả trứng có trước?" là một câu hỏi triết học và sinh học cổ điển, thường được đưa ra để tranh luận về nguyên nhân và hệ quả, cũng như về sự tiến hóa. Để phân tích câu hỏi này, có thể xem xét các khía cạnh sau:\n\n1. **Khái niệm về nguyên nhân và hệ quả**: Câu hỏi đặt ra vấn đề về điều gì đã xảy ra đầu tiên - kết quả (con gà) hay nguyên nhân (quả trứng).\n\n2. **Bố